In [1]:
import numpy as np
from hidden_markov import hmm
from sim2net.packet_loss.gilbert_elliott import GilbertElliott
import random
from sklearn.metrics import mean_squared_error

In [2]:
# States
states = ('G', 'B')

# list of possible observations
possible_observation = (0,1)

# The observations that we observe and feed to the model
obs = []
quantities_observations = []
p_list = []
r_list = []
h_list = []
k_list = []
ppred_list = []
rpred_list = []
hpred_list = []
kpred_list = []
observation = []

for i in range(5):
    p = random.random() #alpha
    r = random.random() #beta
    h = random.random()
    k = random.random()
    p_list.append(p)
    r_list.append(r)
    h_list.append(h)
    k_list.append(k)

In [3]:
for z in range(5):
    p = p_list[z] #alpha
    r = r_list[z] #beta
    h = h_list[z]
    k = k_list[z]
    for i in range(10):
        for j in range(10000):
            ge = GilbertElliott([p,r,h,k])
            pl = int(ge.packet_loss())
            obs.append(pl)

        observation_tuple = tuple(obs)
        observation_list = [observation_tuple]
        observation.extend(observation_list)
        quantities_observations.append(1)
        obs = []
        observation_tuple = ()
        observation_list = []

    # Input parameters as Numpy matrices
    start_probability = np.matrix( '0.5 0.5 ')
    transition_probability = np.matrix('0.6 0.4 ;  0.3 0.7 ')
    emission_probability = np.matrix( '0.3 0.7 ; 0.4 0.6 ' )
    
    test = hmm(states,possible_observation,start_probability,transition_probability,emission_probability)
    
    prob = test.log_prob(observation, quantities_observations)
    #print ("Probability of sequence with original parameters : %f"%(prob))
    
    #Apply Baum-welch Algorithm
    num_iter=1000
    emission,transition,start = test.train_hmm(observation,num_iter,quantities_observations)
    
    #Print output after applying the algorithm
    """print(emission)
    
    print(start)
    
    print(transition) """
    
    prob = test.log_prob(observation, quantities_observations)
    #print ("Probability of sequence after %d iterations : %f"%(num_iter,prob))
    print("Iteration:",z+1)
    p_diff = abs(transition[0,1] - p)
    r_diff = abs(transition[1,0] - r)
    h_diff = abs(emission[1,0] - h)
    k_diff = abs(emission[0,0] - k)
    ppred_list.append(transition[0,1])
    rpred_list.append(transition[1,0])
    hpred_list.append(emission[1,0])
    kpred_list.append(emission[0,0])
    #print()
    quantities_observations = []
    observation = []

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5


In [4]:
print("MSE for p:", mean_squared_error(p_list, ppred_list))
print("MSE for r:", mean_squared_error(r_list, rpred_list))
#ppred_list = []
#rpred_list = []

MSE for p: 0.0966907517107271
MSE for r: 0.061903241460138925


In [5]:
print("MSE for h:", mean_squared_error(h_list, hpred_list))
print("MSE for k:", mean_squared_error(k_list, kpred_list))
#hpred_list = []
#kpred_list = []

MSE for h: 0.028019913620074922
MSE for k: 0.11957025419831269


In [6]:
s = 0.0
z = 0.0
for i in range(5):
    s = s + float((p_list[i] - ppred_list[i])**2)

m = float(s/5)
print(m)
print(float(m/np.mean(p_list)))

0.0966907517107271
0.16268522907336236


In [7]:
p_list

[0.8568663891797068,
 0.24274008079983211,
 0.5816413055294084,
 0.3854663792383528,
 0.9049985281270845]

In [8]:
ppred_list

[0.46849757966443056,
 0.38531011760575606,
 0.4047862543461118,
 0.3846203541862897,
 0.3748864776443163]

In [9]:
print(r_list)
print(rpred_list)

[0.4223562190445217, 0.3591611417536321, 0.012696090261297366, 0.6430069735346503, 0.027388006448417657]
[0.24484265064883354, 0.31311510977015133, 0.2958768539757738, 0.3137183321806077, 0.32279035750878876]


In [10]:
print(h_list)
print(hpred_list)

[0.930293744071802, 0.5937381999091741, 0.2952568015081234, 0.1508192179354063, 0.1702097194246961]
[0.8290276160135368, 0.2942716109674621, 0.433017784117476, 0.28932526291748045, 0.2149588947244339]


In [14]:
print(k_list)
print(kpred_list)

[0.0417951719680677, 0.1478447981586789, 0.5169798825574576, 0.31439363441506996, 0.3371933361781263]
[0.7574864408001483, 0.21137976551243215, 0.32937352090885724, 0.20739575554171052, 0.15022838326442126]


In [15]:
s = 0.0
z = 0.0
for i in range(5):
    s = s + float((r_list[i] - rpred_list[i])**2)

m = float(s/5)
print(m)
print(float(m/np.mean(r_list)))

0.061903241460138925
0.21133034655575395


In [16]:
s = 0.0
z = 0.0
for i in range(5):
    s = s + float((h_list[i] - hpred_list[i])**2)

m = float(s/5)
print(m)
print(float(m/np.mean(h_list)))

0.028019913620074922
0.06545737075529524


In [17]:
s = 0.0
z = 0.0
for i in range(5):
    s = s + float((k_list[i] - kpred_list[i])**2)

m = float(s/5)
print(m)
print(float(m/np.mean(k_list)))

0.11957025419831269
0.44017690144489735
